In [1]:
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import seaborn as sns
import sys
import sys
import re
import os.path

## data import

In [2]:
file_name =  "clean_data_2022-08-27.csv"
folder = "../clean_equity_data/"
save = "YES"

data_tmp = pd.read_csv(folder + file_name)

In [3]:
data_tmp.columns

Index(['Unnamed: 0', 'Börsdata ID', 'Company', 'Industry', 'Volume', 'P/FCF',
       'Assets Turn', 'Gross profit', 'Tot. Assets', 'ROC', 'Market Cap',
       'Volatility', 'ROE', 'OCF', 'Total Equity', 'EV', 'Return 6m',
       'Return 1y', 'Country', 'List', 'Tick', 'EV/EBIT', 'Last Report',
       'Yahoo', 'Yield', 'P/E', 'EV/S', 'P/B', 'Cash.f.Fin.',
       'Info - Stock price', 'Info - Report', 'Sector', 'CF_prof', 'GPA',
       'Financing_Yield', 'EA ret', 'EA ret std', '1 Year Volatility',
       '1 Year MAD', 'EA Volatility', 'Beta', 'Res_Mom_1Y', 'Res_Mom_6m',
       'Total_Mom_6m'],
      dtype='object')

In [4]:
# FUNCTIONS FOR VIEWING SINGLE COMPANY RANKINGS 

def company(company, data):
    for i in data['Company']:
        if company in i or company in i.lower():
            return data.loc[data['Company'] == i]
    print('No company found')
    
def Company(company, data):
    for i in data['Company']:
        if company in i or company in i.lower():
            stock = data.loc[data['Company'] == i]
            stock = stock[compact]
            return stock
    print('No company found')

In [5]:
### GESTALT
data_gestalt = data_tmp.copy()
data_gestalt = data_gestalt.loc[(data_tmp['Sector'] != 'Financials')]

### Gestalt Imputation and Rank

In [6]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to median, only fundamental data
columns = ['P/E', 'EV/EBIT', 'P/FCF', 'EV/S', 'P/B', 'ROC', 'ROE',
            'GPA', 'Assets Turn', 'CF_prof', 'Financing_Yield']
for i in columns: 
    data_gestalt.loc[data_gestalt[i].isna() ,i] = data_gestalt[i].median()
    
# set negative values to max
for i in ['P/E', 'EV/EBIT', 'P/FCF', 'EV/S', 'P/B']:
    data_gestalt.loc[data_gestalt[i] < 0 ,i] = data_gestalt[i].max()
    
# set nan yield & vol to 0
data_gestalt.loc[data_gestalt['Yield'].isna(),'Yield'] = 0

In [7]:
RANK_WEIGHTS = {"Momentum": 1, "Value": 1, "Quality": 1}



factor=1.0/sum(RANK_WEIGHTS.values())
for k in RANK_WEIGHTS:
    RANK_WEIGHTS[k] = RANK_WEIGHTS[k]*factor
    
    
print(sum(RANK_WEIGHTS.values()))

1.0


In [8]:
RANK_WEIGHTS

{'Momentum': 0.3333333333333333,
 'Value': 0.3333333333333333,
 'Quality': 0.3333333333333333}

In [9]:
####### CALCULATE RANKS ###########
for i in ['Total_Mom_6m', 'Return 6m', 'Return 1y', 'EA ret std', 'Res_Mom_1Y', 'Res_Mom_6m']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank(ascending = False)

# Ranking where lower value is better
for i in ['P/E', 'P/B', 'EV/S', 'P/FCF', 'EV/EBIT', 'Financing_Yield']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank()
    

for i in ['Yield', 'ROE', 'ROC', 'CF_prof', 'GPA', 'Assets Turn']:
    data_gestalt[i + ' Rank'] = (data_gestalt[i]).rank(ascending = False)
    
##################### Composite ranks #######################
data_gestalt['PMOM Rank'] = (data_gestalt['Total_Mom_6m Rank'] + data_gestalt['Return 1y Rank']).rank(ascending = True)
data_gestalt['IMOM Rank'] = (data_gestalt['Res_Mom_1Y Rank'] + data_gestalt['Res_Mom_6m Rank']).rank(ascending = True)
data_gestalt['Momentum Rank'] = (data_gestalt['EA ret std Rank'] + 
                                 data_gestalt['IMOM Rank'] + data_gestalt['PMOM Rank']).rank(ascending = True)


data_gestalt['Quality Rank'] = (data_gestalt['ROE Rank'] + data_gestalt['ROC Rank'] + data_gestalt['CF_prof Rank'] + 
                        data_gestalt['GPA Rank']).rank()

data_gestalt['Value Rank'] = (data_gestalt['P/E Rank'] + data_gestalt['P/B Rank'] + data_gestalt['EV/S Rank'] + data_gestalt['P/FCF Rank'] + 
                      data_gestalt['EV/EBIT Rank'] + data_gestalt['Financing_Yield Rank']).rank()
 


    
data_gestalt['Gestalt Rank'] = ((RANK_WEIGHTS['Value']*data_gestalt['Value Rank'] 
                                 + RANK_WEIGHTS['Momentum']*data_gestalt['Momentum Rank'] +
                                RANK_WEIGHTS['Quality']* data_gestalt['Quality Rank'])).rank()



### Data Error Check

In [10]:
#data_mom.isna().sum()
data_gestalt.isna().sum().sum()

50

### Sorting

In [11]:
CURRENT_BUYOUTS = ['Aspire Global', 'LeoVegas', 'Swedish Match', 'Atvexa', 'Semcon']

#### Gestalt Data Frame
gestalt_tmp = data_gestalt.sort_values(by=['Gestalt Rank'])
gestalt_tmp=gestalt_tmp[ ~gestalt_tmp['Company'].isin(CURRENT_BUYOUTS)]


In [12]:

if save == "YES":
    data_date  = re.search(r'\d{4}-\d{2}-\d{2}', file_name)
    folder = '../clean_equity_data'
    exp_file_name = "GESTALT_{}".format(data_date.group(0))+ ".csv"
    gestalt_tmp.to_csv(os.path.join(folder, exp_file_name))

In [15]:
compact = ['Company', 'List','Tick', 'Gestalt Rank', 'Quality Rank', 'Value Rank', 'Momentum Rank', 'EA ret std', '1 Year Volatility']
compact_mom = ['Company', 'List','Tick','Momentum Rank','EA ret std', '1 Year Volatility']
# numbers of stocks to select for diff strategies

stocks = 50


## TRIPLE SORT ##

gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

### Print Stock Selection

In [16]:
#gestalt 
th_props = [
  ('font-size', '14px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]
# Set CSS properties for td elements in dataframe
td_props = [('font-size', '12px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]
# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]
gestalt.index = range(1, len(gestalt)+1)
gestalt_view = (gestalt.style.apply(lambda x: ['background: lightgreen' if x.name in range(1, 15+1) 
                              else '' for i in x], axis=1).set_table_styles(styles)
                 .format({'Gestalt Rank': "{:.0f}",'Quality Rank': "{:.0f}", 'Value Rank': "{:.0f}",
                          'Momentum Rank': "{:.0f}",
                          'EA ret std': "{:.1f}",'1 Year Volatility': "{:.1%}"})
                 .set_caption("Gestalt"))
gestalt_view

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
1,Arctic Paper,Small Cap,ARP,1,43,4,5,1.0,55.5%
2,Dedicare,Small Cap,DEDI,2,10,33,10,2.9,42.4%
3,Rottneros,Mid Cap,RROS,3,46,3,9,1.6,45.0%
4,International Petroleum,Mid Cap,IPCO,4,39,27,18,-0.1,46.1%
5,SSAB B,Large Cap,SSAB B,5,50,5,35,-0.1,41.8%
6,EnQuest,Mid Cap,ENQ,6,80,15,8,1.9,61.5%
7,B3 Consulting,Small Cap,B3,7,12,47,46,-0.6,53.1%
8,New Wave,Mid Cap,NEWA B,8,68,40,4,3.5,51.3%
9,Betsson,Mid Cap,BETS B,9,62,36,14,3.5,41.1%
10,Prevas,Small Cap,PREV B,10,18,31,66,0.0,51.4%


## Reversed Ranks

In [14]:
stocks = 20

## TRIPLE SORT ##
gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

neg_gestalt = gestalt_tmp.tail(stocks)[::-1][compact]

In [16]:
gestalt_tmp.sort_values(by = 'EA ret std').tail(30)[['Company','Res_Mom_1Y', 'EA ret std', 'Financing_Yield','Momentum Rank']]


,Company,Res_Mom_1Y,EA ret std,Financing_Yield,Momentum Rank
128,Alfa Laval,1.758000,2.154281,-0.035277,52.0
261,I-Tech,-3.396093,2.159716,-0.031992,150.0
175,Hanza Holding,13.081608,2.168849,0.031061,29.0
108,Holmen B,2.515741,2.205225,-0.046033,20.0
222,Truecaller,-2.129770,2.349742,0.366652,149.0
47,TietoEVRY,5.605046,2.420283,-0.073430,19.0
251,Viaplay B,-16.416172,2.445933,0.001941,183.5
146,Nokia,18.522635,2.555549,-0.021254,12.0
186,Addnode,0.473511,2.620378,-0.009864,38.0
365,Medivir,9.825278,2.629392,-0.007584,34.0


In [17]:
neg_gestalt

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
293,ExpreS2ion Biotech,First North,EXPRS2,305.0,292.0,294.0,258.0,0.059502,0.837954
245,Nanoform,First North,NANOFS,304.0,285.5,281.0,272.0,-0.749432,0.409234
296,SynAct Pharma,Mid Cap,SYNACT,303.0,273.5,290.5,270.0,-0.461925,0.779819
241,Re:NewCell,First North,RENEW,302.0,282.0,296.0,244.5,-0.891352,0.579075
254,Azelio,First North,AZELIO,301.0,290.0,233.0,299.0,-1.292725,1.068817
322,OptiCept Technologies,First North,OPTI,300.0,278.0,274.5,263.0,-0.347838,0.571101
266,Acconeer,First North,ACCON,299.0,281.0,303.0,223.0,-0.668314,0.684515
303,Immunovia,Mid Cap,IMMNOV,298.0,300.0,249.0,254.0,-0.622859,0.963201
268,IRRAS,Small Cap,IRRAS,297.0,304.0,304.0,194.0,1.874874,2.293349
347,Episurf Medical,Small Cap,EPIS B,296.0,293.5,287.0,216.0,0.688558,0.592213


In [17]:

if save == "YES":
    data_date  = re.search(r'\d{4}-\d{2}-\d{2}', file_name)
    folder = '../clean_equity_data'
    exp_file_name = "rank_data_{}".format(data_date.group(0))+ ".csv"
    data_tmp.to_csv(os.path.join(folder, exp_file_name))

In [18]:
company('EnQuest', gestalt_tmp)

,Unnamed: 0,Börsdata ID,Company,Industry,Volume,P/FCF,Assets Turn,Gross profit,Tot. Assets,ROC,...,ROC Rank,CF_prof Rank,GPA Rank,Assets Turn Rank,PMOM Rank,IMOM Rank,Momentum Rank,Quality Rank,Value Rank,Gestalt Rank
372,372,75,EnQuest,Oil & Gas -Exploration,15.542,2.069,0.29,3463.601,42215.672,0.18789,...,87.0,42.0,263.0,243.5,24.0,32.0,8.0,79.5,15.0,6.0


In [19]:
gestalt_tmp[['Momentum Rank', 'Quality Rank', 'Value Rank', 'Gestalt Rank']].corr(method = 'spearman')

,Momentum Rank,Quality Rank,Value Rank,Gestalt Rank
Momentum Rank,1.000000,0.133048,0.059653,0.541157
Quality Rank,0.133048,1.000000,0.597268,0.809161
Value Rank,0.059653,0.597268,1.000000,0.771028
Gestalt Rank,0.541157,0.809161,0.771028,1.000000


In [21]:
gestalt_tmp.sort_values(by = 'Financing_Yield Rank')

,Unnamed: 0,Börsdata ID,Company,Industry,Volume,P/FCF,Assets Turn,Gross profit,Tot. Assets,ROC,...,ROC Rank,CF_prof Rank,GPA Rank,Assets Turn Rank,PMOM Rank,IMOM Rank,Momentum Rank,Quality Rank,Value Rank,Gestalt Rank
151,151,132,Orrön Energy,Renewable Energy,577.621,0.329,6.901,19958.850,3797.800,13.09457,...,1.0,1.0,1.0,1.0,301.0,218.0,241.0,1.0,2.0,41.0
212,212,2383,Momentum Group,Industrial Components,1.207,33.143,1.563,761.000,1017.000,0.26282,...,51.0,63.0,17.0,22.0,304.0,162.5,208.0,22.0,97.0,84.0
5,5,38,Björn Borg,Clothing & Footwear,1.233,4.686,1.345,456.369,601.590,0.27059,...,47.0,15.0,15.0,39.0,174.0,217.0,264.0,14.0,11.0,63.0
210,210,2401,Engcon,Industrial Components,13.991,77.828,1.615,713.000,1059.000,0.88834,...,9.0,31.0,27.0,19.0,304.0,162.5,229.0,8.0,172.0,120.0
33,33,222,Tele2 B,Telecommunications,229.999,5.033,0.411,11047.000,66305.000,0.12344,...,134.0,46.0,220.0,221.0,80.5,50.5,114.0,97.0,80.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,332,387,Cell Impact,Industrial Machinery,3.419,4194.257,0.231,19.399,433.449,-0.49273,...,277.0,283.0,269.0,255.0,244.0,247.0,218.0,285.5,289.0,292.0
331,331,398,Cortus Energy,Biomass Energy,2.454,4194.257,0.020,-27.926,110.546,-1.30068,...,293.0,297.0,303.0,295.0,144.0,100.5,73.0,303.0,305.0,262.0
242,242,2023,CDON,Retailers,0.796,4194.257,1.860,237.900,270.900,-2.96491,...,301.0,298.0,11.0,10.0,154.0,162.5,86.5,242.5,299.0,244.0
292,292,1361,Filo Mining,Mining,3.845,4194.257,0.710,-401.728,819.760,0.10611,...,153.0,279.0,304.0,153.0,4.0,42.0,47.0,262.5,292.0,233.0
